In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
dl_tr = pd.read_csv('//content/drive/My Drive/train.csv')
dl_te = pd.read_csv('//content/drive/My Drive/test.csv')

In [ ]:
import nltk
dl_tr = dl_tr.drop(columns = 'ID')
dl_te = dl_te.drop(columns = 'ID')
import nltk
import re
from nltk.corpus import stopwords
dl_tr.select_dtypes(int).columns

Index(['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance'],
      dtype='object')

In [ ]:
a = dl_tr['TITLE'].map(lambda x:re.sub('[^ζωσμπδαβγχλκτεψηξφθν]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)

b = dl_tr['TITLE'].map(lambda x:re.sub('[^$]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)

c = dl_tr['TITLE'].map(lambda x:re.sub('[^+=*/><%]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)

d = dl_tr['TITLE'].map(lambda x:re.sub('[^{}]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)

e = dl_tr['ABSTRACT'].map(lambda x:re.sub('[^$]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)

f = dl_tr['ABSTRACT'].map(lambda x:re.sub('[^+=*/><%]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)

g = dl_tr['ABSTRACT'].map(lambda x:re.sub('[^{}]',' ',x)).map(lambda x:''.join(x.split())).replace('',np.nan).notnull().astype(int)



In [ ]:
 nltk.download('stopwords')
 nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = WordNetLemmatizer()
corpus = []
morpus = []
for i in range(0, len(dl_tr)):
    review = ' '.join([ps.lemmatize(word) for word in  re.sub('[^a-zA-Z]', ' ', dl_tr['TITLE'][i]).lower().split() if not word in stopwords.words('english')])
    review1 = ' '.join([ps.lemmatize(word) for word in  re.sub('[^a-zA-Z]', ' ', dl_tr['ABSTRACT'][i]).lower().split() if not word in stopwords.words('english')])
    corpus.append(review)
    morpus.append(review1)

In [ ]:
voc_size = 5000
corpus_1 =[one_hot(words,voc_size)for words in corpus] 
morpus_1 = [one_hot(words,voc_size)for words in morpus] 

In [ ]:
sent_length_1 = 27
sent_length_2 = 200
embedded_docs_1=pad_sequences(corpus_1,padding='pre',maxlen=sent_length_1)
embedded_docs_2=pad_sequences(morpus_1,padding='pre',maxlen=sent_length_2)
print(embedded_docs_2)
print(embedded_docs_2.shape)

[[   0    0    0 ... 2788 1397 2141]
 [   0    0    0 ... 1896 1520 2459]
 [   0    0    0 ... 3788 1115 4538]
 ...
 [   0    0    0 ...  191  546 3335]
 [   0    0    0 ... 1720 1510 3870]
 [   0    0    0 ... 1394 4158 1260]]
(20972, 200)


In [ ]:
add_1 = np.array(pd.DataFrame([a,b,c,d]).T)
add_2 = np.array(pd.DataFrame([e,f,g]).T)

In [ ]:
embedding_vector_features_1 = 50
embedding_vector_features_2 = 50

In [ ]:
import keras
from keras.models import Model
from keras.layers import Concatenate
from keras.layers import Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Dropout

input1 = keras.Input(shape=(sent_length_1,))
embed1 = Embedding(voc_size, embedding_vector_features_1, input_length= sent_length_1 )(input1)
drop_a = Dropout(0.1)(embed1)
lstm1 = LSTM(100)(drop_a)
drop_b = Dropout(0.2)(lstm1)
key1 = keras.Input(shape=(4,))
kala1 = Dense(3,activation= 'relu')(key1)
conc1 = Concatenate(axis = 1)([drop_b,kala1])

input2 = keras.Input(shape=(sent_length_2,))
embed2 = Embedding(voc_size, embedding_vector_features_2, input_length= sent_length_2 )(input2)
drop_c = Dropout(0.1)(embed2)
lstm2 = LSTM(100)(drop_c)
drop_d = Dropout(0.2)(lstm2)
key2 = keras.Input(shape=(3,))
kala2 = Dense(3,activation= 'relu')(key2)
conc2 = Concatenate(axis = 1)([drop_d,kala2])

all_mix = Concatenate(axis = 1)([conc1,conc2])
dens = Dense(6,activation = 'sigmoid')(all_mix)
model = Model(inputs = [input1, key1 , input2 , key2],outputs = dens)
model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics = ['accuracy','AUC'])

In [ ]:
import numpy as np
X_final_1= np.array(embedded_docs_1)
X_final_2 = np.array(embedded_docs_2)
y_final=np.array(dl_tr.select_dtypes(int))

In [ ]:
mixi_1 = np.concatenate([X_final_1,add_1], axis = 1)
mixi_2 =  np.concatenate([X_final_2,add_2], axis = 1)
fruit = np.concatenate([mixi_1,mixi_2], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fruit, y_final, test_size=0.10, random_state= 100)

In [ ]:
one = X_train[:,:27]
two = X_train[:,27:27+4]
three = X_train[:,27+4:27+4+sent_length_2]
four = X_train[:,27+4+sent_length_2:27+4+sent_length_2+3]

one_1 = X_test[:,:27]
two_1 = X_test[:,27:27+4]
three_1 = X_test[:,27+4:27+4+sent_length_2]
four_1 = X_test[:,27+4+sent_length_2:27+4+sent_length_2+3]


In [ ]:
with tf.device("/gpu:0"): 
  model.fit([one,two,three,four],y_train, validation_data=([one_1,two_1,three_1,four_1],y_test),epochs = 2,batch_size= 32 , verbose = 1)

Epoch 1/2
590/590 [==============================] - 41s 69ms/step - loss: 0.3045 - accuracy: 0.6796 - auc: 0.9006 - val_loss: 0.2315 - val_accuracy: 0.7092 - val_auc: 0.9457
Epoch 2/2
590/590 [==============================] - 40s 67ms/step - loss: 0.2048 - accuracy: 0.7677 - auc: 0.9564 - val_loss: 0.2112 - val_accuracy: 0.7550 - val_auc: 0.9554


In [ ]:
out_1 = pd.DataFrame(model.predict([one_1,two_1,three_1,four_1]))

In [ ]:
out_2 = out_1.apply(lambda x:x.map(lambda y:0 if y < 0.5 else 1))

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,out_2, average= 'micro')

0.7876899104012466

In [ ]:
# SHOULD PREDICT ON THE TEST DATA AFTER PERFORMING NECESSARY TRANSFROMATIONS